In [15]:
# organize imports
import cv2
import imutils
import numpy as np

# global variables
bg = None

#-------------------------------------------------------------------------------
# Function - To find the running average over the background
#-------------------------------------------------------------------------------
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

#-------------------------------------------------------------------------------
# Function - To segment the region of hand in the image
#-------------------------------------------------------------------------------
def segment(image, threshold=17):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff,
                                threshold,
                                255,
                                cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (_, cnts, _) = cv2.findContours(thresholded.copy(),
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

#-------------------------------------------------------------------------------
# Main function
#-------------------------------------------------------------------------------
if __name__ == "__main__":
    # initialize weight for running average
    aWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0

    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width=700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand
                print(segmented)
                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                cv2.imwrite(str(num_frames)+'.png',thresholded)
                cv2.imshow("Thesholded", thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break
            

# free up memory
camera.release()
cv2.destroyAllWindows()

[[[168 136]]

 [[167 137]]

 [[168 138]]

 [[170 138]]

 [[170 137]]

 [[169 137]]]
[[[120 144]]]
[[[75 90]]]
[[[191  81]]

 [[190  82]]

 [[191  82]]]
[[[124 152]]

 [[125 153]]

 [[125 152]]]
[[[104 152]]

 [[106 152]]]
[[[ 72 210]]

 [[ 71 211]]

 [[ 70 211]]

 [[ 69 212]]

 [[ 68 212]]

 [[ 67 213]]

 [[ 67 214]]

 [[ 84 214]]

 [[ 84 213]]

 [[ 83 212]]

 [[ 82 212]]

 [[ 81 211]]

 [[ 80 211]]

 [[ 79 210]]]
[[[ 73 198]]

 [[ 72 199]]

 [[ 70 199]]

 [[ 65 204]]

 [[ 64 203]]

 [[ 62 203]]

 [[ 61 202]]

 [[ 54 202]]

 [[ 53 203]]

 [[ 52 203]]

 [[ 51 204]]

 [[ 50 204]]

 [[ 49 205]]

 [[ 49 206]]

 [[ 48 207]]

 [[ 48 208]]

 [[ 47 209]]

 [[ 47 210]]

 [[ 46 211]]

 [[ 46 214]]

 [[102 214]]

 [[102 213]]

 [[101 212]]

 [[101 211]]

 [[100 211]]

 [[ 99 210]]

 [[ 95 210]]

 [[ 94 209]]

 [[ 89 209]]

 [[ 86 206]]

 [[ 86 205]]

 [[ 85 204]]

 [[ 85 203]]

 [[ 82 200]]

 [[ 81 200]]

 [[ 80 199]]

 [[ 78 199]]

 [[ 77 198]]]
[[[ 72 185]]

 [[ 71 186]]

 [[ 70 186]]

 [[ 68 1

[[[156  97]]

 [[155  98]]

 [[154  98]]

 [[153  99]]

 [[151  99]]

 [[150 100]]

 [[149 100]]

 [[148 101]]

 [[147 101]]

 [[144 104]]

 [[143 104]]

 [[142 105]]

 [[141 105]]

 [[140 106]]

 [[138 106]]

 [[137 105]]

 [[135 105]]

 [[134 106]]

 [[132 106]]

 [[131 107]]

 [[129 107]]

 [[128 108]]

 [[126 108]]

 [[125 109]]

 [[123 109]]

 [[118 114]]

 [[117 114]]

 [[112 119]]

 [[112 120]]

 [[109 123]]

 [[109 124]]

 [[107 126]]

 [[107 127]]

 [[105 129]]

 [[105 130]]

 [[103 132]]

 [[103 133]]

 [[102 134]]

 [[102 135]]

 [[101 136]]

 [[101 137]]

 [[100 138]]

 [[100 140]]

 [[ 98 142]]

 [[ 98 143]]

 [[ 97 144]]

 [[ 97 146]]

 [[ 96 147]]

 [[ 96 148]]

 [[ 95 149]]

 [[ 95 150]]

 [[ 94 151]]

 [[ 94 152]]

 [[ 93 153]]

 [[ 93 154]]

 [[ 92 155]]

 [[ 92 157]]

 [[ 91 158]]

 [[ 91 159]]

 [[ 90 160]]

 [[ 90 161]]

 [[ 89 162]]

 [[ 89 163]]

 [[ 88 164]]

 [[ 88 166]]

 [[ 87 167]]

 [[ 87 168]]

 [[ 86 169]]

 [[ 86 170]]

 [[ 85 171]]

 [[ 85 172]]

 [[ 84

[[[194  47]]

 [[193  48]]

 [[190  48]]

 [[189  49]]

 [[186  49]]

 [[185  50]]

 [[183  50]]

 [[182  51]]

 [[181  51]]

 [[180  52]]

 [[179  52]]

 [[177  54]]

 [[167  54]]

 [[166  55]]

 [[161  55]]

 [[160  56]]

 [[158  56]]

 [[157  57]]

 [[156  57]]

 [[155  58]]

 [[154  58]]

 [[153  59]]

 [[152  59]]

 [[150  61]]

 [[149  61]]

 [[144  66]]

 [[143  66]]

 [[135  74]]

 [[135  75]]

 [[133  77]]

 [[133  78]]

 [[131  80]]

 [[131  81]]

 [[130  82]]

 [[130  83]]

 [[129  84]]

 [[129  85]]

 [[127  87]]

 [[127  88]]

 [[126  89]]

 [[126  90]]

 [[124  92]]

 [[124  93]]

 [[123  94]]

 [[123  95]]

 [[122  96]]

 [[122  97]]

 [[119 100]]

 [[119 101]]

 [[118 102]]

 [[118 103]]

 [[117 104]]

 [[117 105]]

 [[116 106]]

 [[116 107]]

 [[115 108]]

 [[115 109]]

 [[114 110]]

 [[114 112]]

 [[113 113]]

 [[113 115]]

 [[112 116]]

 [[112 118]]

 [[111 119]]

 [[111 121]]

 [[110 122]]

 [[110 123]]

 [[109 124]]

 [[109 126]]

 [[108 127]]

 [[108 129]]

 [[107

[[[204  34]]

 [[203  35]]

 [[202  35]]

 [[201  36]]

 [[199  36]]

 [[197  38]]

 [[196  38]]

 [[195  39]]

 [[194  39]]

 [[193  40]]

 [[191  40]]

 [[190  41]]

 [[189  41]]

 [[188  40]]

 [[184  40]]

 [[183  39]]

 [[182  40]]

 [[176  40]]

 [[175  41]]

 [[173  41]]

 [[172  42]]

 [[171  42]]

 [[170  43]]

 [[169  43]]

 [[168  44]]

 [[167  44]]

 [[166  45]]

 [[165  45]]

 [[163  47]]

 [[162  47]]

 [[157  52]]

 [[156  52]]

 [[152  56]]

 [[151  56]]

 [[149  58]]

 [[149  59]]

 [[145  63]]

 [[141  63]]

 [[140  62]]

 [[139  63]]

 [[137  63]]

 [[136  64]]

 [[136  65]]

 [[135  66]]

 [[135  67]]

 [[134  68]]

 [[134  69]]

 [[133  70]]

 [[133  73]]

 [[132  74]]

 [[132  77]]

 [[131  78]]

 [[131  79]]

 [[130  80]]

 [[130  82]]

 [[129  83]]

 [[129  84]]

 [[127  86]]

 [[127  87]]

 [[126  88]]

 [[126  89]]

 [[125  90]]

 [[125  91]]

 [[124  92]]

 [[124  93]]

 [[122  95]]

 [[122  96]]

 [[121  97]]

 [[121  98]]

 [[120  99]]

 [[120 100]]

 [[119

[[[221  34]]

 [[220  35]]

 [[218  35]]

 [[217  36]]

 [[216  36]]

 [[213  39]]

 [[212  39]]

 [[210  41]]

 [[209  41]]

 [[206  44]]

 [[205  44]]

 [[204  43]]

 [[204  40]]

 [[203  39]]

 [[203  38]]

 [[202  37]]

 [[201  37]]

 [[200  36]]

 [[195  36]]

 [[194  37]]

 [[193  37]]

 [[192  38]]

 [[191  38]]

 [[190  39]]

 [[189  39]]

 [[188  40]]

 [[187  40]]

 [[185  42]]

 [[184  42]]

 [[182  44]]

 [[181  44]]

 [[180  45]]

 [[180  46]]

 [[179  47]]

 [[178  47]]

 [[173  52]]

 [[172  52]]

 [[165  59]]

 [[164  59]]

 [[160  55]]

 [[159  55]]

 [[158  54]]

 [[156  54]]

 [[153  57]]

 [[153  59]]

 [[152  60]]

 [[152  62]]

 [[151  63]]

 [[151  64]]

 [[150  65]]

 [[150  67]]

 [[149  68]]

 [[149  69]]

 [[148  70]]

 [[148  71]]

 [[146  73]]

 [[146  74]]

 [[145  75]]

 [[145  76]]

 [[143  78]]

 [[143  79]]

 [[142  80]]

 [[142  81]]

 [[139  84]]

 [[139  85]]

 [[138  86]]

 [[138  87]]

 [[136  89]]

 [[136  90]]

 [[132  94]]

 [[132  95]]

 [[131

...

[[[163   0]]

 [[163   2]]

 [[162   3]]

 [[162   4]]

 [[161   5]]

 [[161   6]]

 [[158   9]]

 [[158  10]]

 [[156  12]]

 [[156  13]]

 [[155  14]]

 [[155  15]]

 [[152  18]]

 [[152  19]]

 [[151  20]]

 [[150  20]]

 [[146  24]]

 [[146  25]]

 [[145  26]]

 [[145  27]]

 [[144  28]]

 [[144  29]]

 [[143  30]]

 [[143  31]]

 [[142  32]]

 [[142  33]]

 [[141  34]]

 [[141  35]]

 [[140  36]]

 [[140  37]]

 [[139  38]]

 [[139  40]]

 [[138  41]]

 [[138  42]]

 [[137  43]]

 [[137  53]]

 [[136  54]]

 [[136  59]]

 [[137  60]]

 [[137  63]]

 [[136  64]]

 [[136  73]]

 [[135  74]]

 [[135  76]]

 [[134  77]]

 [[134  81]]

 [[133  82]]

 [[133  85]]

 [[132  86]]

 [[132  88]]

 [[131  89]]

 [[131  93]]

 [[130  94]]

 [[130  96]]

 [[129  97]]

 [[129  98]]

 [[121 106]]

 [[120 106]]

 [[118 108]]

 [[118 109]]

 [[116 111]]

 [[116 112]]

 [[114 114]]

 [[114 115]]

 [[113 116]]

 [[113 117]]

 [[107 123]]

 [[106 123]]

 [[103 126]]

 [[103 127]]

 [[ 97 133]]

 [[ 96

[[[176   0]]

 [[175   1]]

 [[175   3]]

 [[174   4]]

 [[174   5]]

 [[171   8]]

 [[171   9]]

 [[168  12]]

 [[168  13]]

 [[161  20]]

 [[161  21]]

 [[160  22]]

 [[160  23]]

 [[158  25]]

 [[158  26]]

 [[157  27]]

 [[157  28]]

 [[155  30]]

 [[155  32]]

 [[153  34]]

 [[153  35]]

 [[151  37]]

 [[151  38]]

 [[148  41]]

 [[148  42]]

 [[146  44]]

 [[146  46]]

 [[145  47]]

 [[145  48]]

 [[144  49]]

 [[144  50]]

 [[143  51]]

 [[143  52]]

 [[142  53]]

 [[142  59]]

 [[143  60]]

 [[143  64]]

 [[144  65]]

 [[144  68]]

 [[145  69]]

 [[145  71]]

 [[146  72]]

 [[146  75]]

 [[147  76]]

 [[147  78]]

 [[146  79]]

 [[146  82]]

 [[145  83]]

 [[145  85]]

 [[144  86]]

 [[144  87]]

 [[143  88]]

 [[143  90]]

 [[142  91]]

 [[142  93]]

 [[141  94]]

 [[141  95]]

 [[140  96]]

 [[140  97]]

 [[139  98]]

 [[139  99]]

 [[138 100]]

 [[138 102]]

 [[136 104]]

 [[136 105]]

 [[131 110]]

 [[130 110]]

 [[128 112]]

 [[127 112]]

 [[122 117]]

 [[122 118]]

 [[121

[[[175   0]]

 [[175   2]]

 [[173   4]]

 [[173   5]]

 [[172   6]]

 [[172   7]]

 [[171   8]]

 [[171   9]]

 [[170  10]]

 [[170  11]]

 [[168  13]]

 [[168  14]]

 [[167  15]]

 [[167  16]]

 [[160  23]]

 [[160  24]]

 [[157  27]]

 [[157  28]]

 [[156  29]]

 [[156  30]]

 [[154  32]]

 [[154  33]]

 [[152  35]]

 [[152  36]]

 [[149  39]]

 [[149  40]]

 [[147  42]]

 [[147  43]]

 [[146  44]]

 [[146  45]]

 [[144  47]]

 [[144  48]]

 [[143  49]]

 [[143  50]]

 [[142  51]]

 [[142  54]]

 [[143  55]]

 [[143  58]]

 [[144  59]]

 [[144  61]]

 [[145  62]]

 [[145  65]]

 [[146  66]]

 [[146  67]]

 [[147  68]]

 [[147  69]]

 [[148  70]]

 [[148  77]]

 [[147  78]]

 [[147  79]]

 [[146  80]]

 [[146  82]]

 [[145  83]]

 [[145  84]]

 [[144  85]]

 [[144  87]]

 [[143  88]]

 [[143  89]]

 [[142  90]]

 [[142  91]]

 [[141  92]]

 [[141  94]]

 [[140  95]]

 [[140  96]]

 [[139  97]]

 [[139  98]]

 [[138  99]]

 [[138 100]]

 [[131 107]]

 [[130 107]]

 [[128 109]]

 [[127

[[[178   0]]

 [[178   1]]

 [[176   3]]

 [[176   4]]

 [[175   5]]

 [[175   6]]

 [[174   7]]

 [[174   8]]

 [[172  10]]

 [[172  11]]

 [[170  13]]

 [[170  14]]

 [[169  15]]

 [[169  16]]

 [[161  24]]

 [[161  25]]

 [[159  27]]

 [[159  28]]

 [[156  31]]

 [[156  32]]

 [[154  34]]

 [[154  35]]

 [[151  38]]

 [[151  39]]

 [[148  42]]

 [[148  43]]

 [[147  44]]

 [[147  45]]

 [[145  47]]

 [[145  48]]

 [[144  49]]

 [[144  55]]

 [[145  56]]

 [[145  58]]

 [[146  59]]

 [[146  61]]

 [[147  62]]

 [[147  63]]

 [[148  64]]

 [[148  65]]

 [[149  66]]

 [[149  68]]

 [[150  69]]

 [[150  75]]

 [[149  76]]

 [[149  78]]

 [[148  79]]

 [[148  80]]

 [[147  81]]

 [[147  83]]

 [[146  84]]

 [[146  85]]

 [[145  86]]

 [[145  88]]

 [[143  90]]

 [[143  92]]

 [[142  93]]

 [[142  94]]

 [[141  95]]

 [[141  96]]

 [[140  97]]

 [[140  98]]

 [[133 105]]

 [[132 105]]

 [[123 114]]

 [[123 115]]

 [[122 116]]

 [[122 117]]

 [[114 125]]

 [[113 125]]

 [[110 128]]

 [[110

[[[183   0]]

 [[181   2]]

 [[181   3]]

 [[179   5]]

 [[179   6]]

 [[178   7]]

 [[178   8]]

 [[176  10]]

 [[176  11]]

 [[174  13]]

 [[174  15]]

 [[169  20]]

 [[168  20]]

 [[165  23]]

 [[165  24]]

 [[162  27]]

 [[162  28]]

 [[158  32]]

 [[158  33]]

 [[157  34]]

 [[157  35]]

 [[152  40]]

 [[152  41]]

 [[150  43]]

 [[150  44]]

 [[149  45]]

 [[149  46]]

 [[147  48]]

 [[147  55]]

 [[148  56]]

 [[148  58]]

 [[149  59]]

 [[149  62]]

 [[150  63]]

 [[150  64]]

 [[151  65]]

 [[151  66]]

 [[152  67]]

 [[152  71]]

 [[153  72]]

 [[153  73]]

 [[152  74]]

 [[152  77]]

 [[151  78]]

 [[151  79]]

 [[150  80]]

 [[150  81]]

 [[149  82]]

 [[149  83]]

 [[148  84]]

 [[148  86]]

 [[147  87]]

 [[147  88]]

 [[146  89]]

 [[146  91]]

 [[145  92]]

 [[145  93]]

 [[144  94]]

 [[144  95]]

 [[143  96]]

 [[143  97]]

 [[141  99]]

 [[141 100]]

 [[137 104]]

 [[136 104]]

 [[134 106]]

 [[133 106]]

 [[127 112]]

 [[127 113]]

 [[124 116]]

 [[124 117]]

 [[123

In [5]:
camera.release()
cv2.destroyAllWindows

<function destroyAllWindows>

In [ ]:
#the next part of this project is saved under the python file named "handgesture_detect.py" .run it using py2.7